# Hardware specifications

*What you will learn:*
- what is a Device and why is a Device needed;
- what are the available devices and where to find them;
- what do a Device and a Channel enforce; 
- tips on how to pick a `Device`;

## The `Device`

As presented in the [introduction to programming a neutral-atom QPU](programming.md#writing-a-pulser-program), the first step to writing a Pulser program is [the selection of a Device](programming.md#pick-a-device). 

The `Device` is an object that stores **all** the physical constraints a quantum program written with Pulser should verify. The `Device` enforces that each operation added to the quantum program (i.e. the `pulser.Sequence`) respects its constraints.  

<div class="alert alert-info">

**Important note**:


The `Device` represents the physics of a neutral-atom QPU but is not the QPU itself. A QPU must be accessed via a `Backend`, which is presented [in this section](./tutorials/backends.nblink).

</div>

### Choosing a `Device`

<center>
<img src="files/decision_diagram_device.png" alt="Decision Diagram to select a Device for the computation" width="600">
</center>


To choose a `Device` for your `Sequence`, the first question you should ask yourself is "How close do I want the physical constraints I program with to be to the one of a QPU ?" ?

**If you want to program with the physical constraints of a QPU**: Each QPU has an associated `Device`, which you can [get from the cloud provider you use to access this QPU](tutorials/backends.nblink#1.2.-Preparation-for-execution-on-QPUBackend).

There are several reasons for which you might want to feel less constrained by the features currently supported by real QPUs. For instance, you might want to design an algorithm for a QPU having better performances (supporting more qubits, longer sequences, ...) or hardware components that have not been installed.

Pulser enables you to define your own devices, but a `Device` object takes as input lots of parameters that have all to be defined. Therefore, for user convenience, `Pulser` provides:

- **Examples of typical physical devices** in `pulser.devices`. Notably, `pulser.AnalogDevice` is an example of a QPU implementing an [Ising Hamiltonian](./programming.md#ising-hamiltonian).

- **The possibility to define a device without some physical constraints** using the `VirtualDevice` class. An example of such a virtual device is the `MockDevice` provided in the `pulser.devices`, which gives full liberty to write a quantum program. `VirtualDevice` is detailed in [an advanced tutorial](./tutorials/virtual_devices.nblink).

<div class="alert alert-info">

**Note**:

The selection of a device in a Pulser program does not enforce any choice on the [backend](tutorials/backends.nblink). No matter the device you used to program your `Sequence`, you can always submit it to any QPU: if the values of the `Sequence` match the constraints of the `QPU`, it will be executed. 

**Tip**:

It is possible to change the device with which a `Sequence` was built, by using `Sequence.switch_device`. This is especially useful to check if the values of the `Sequence` match the constraints of the `QPU` prior to submitting to the `QPU` (for instance, you could have built your `Sequence` with an example of a `Device` like `AnalogDevice`, and now want to run it on a QPU, or the specifications of your QPU might have changed between your design and submission).

</div>

### Reading through the `Device`'s specifications

The second question you should ask yourself to choose your `Device` is "Do its constraints allow me to program my `Sequence` ?". The device specifications are here to guide your choice:

In [ ]:
import pulser

print(pulser.AnalogDevice.specs)

The `Device` is going to constrain the next steps of your [quantum program](./programming.md#writing-a-pulser-program):

1) "Register" and "Layout" parameters are going to constrain the creation of your `Register`, that is, the number of atoms you can use and how you can place them in place. [As a reminder](programming.md#create-the-register), this impacts the [interaction strength in the interaction Hamiltonian](programming.md#interaction-hamiltonian). The creation of a `Register` is presented here. If the `Device` requires a `Layout` (as in the example here), then you have to associate a `RegisterLayout` to the `Register`, which adds more constraints. Check [this tutorial](./tutorials/reg_layouts.nblink) to see how to do it.

2) Among the "Device" parameters, the "Rydberg level" is going to complete the determination of the [interaction strength in the interaction Hamiltonian](programming.md#interaction-hamiltonian). This level determines the "Ising interaction coefficient", which is the $C_6$ coefficient of the [Ising Hamiltonian](./programming.md#ising-hamiltonian). The quantity $\frac{C_6}{\hbar}$ is accessible via `pulser.AnalogDevice.interaction_coeff`.

3) The "Channels" are going to determine what [Channels are available for the computation](programming.md#pick-the-channels). The type of the available channels are going to define the "supported bases" and "supported states" among the Device parameters. Knowing what states you want to use in your computation, you can first check that they are among the "supported states", then find the bases and their associated channel that enable to use these states using [the conventions page](conventions.md#bases).

4) The "Maximum sequence duration" constrains the duration of the [Pulses you can add](programming.md#add-the-pulses), and therefore the Hamiltonian describing the system can at most be defined between 0 and this value.

5) The "Maximum number of runs" limits the number of runs a quantum program can be executed on the QPU. See the section on `Backends` to read more about this.

<div class="alert alert-info">

**Note**:

Other parameters are being defined in the `Device`. You can see the effect of each them in the [API documentation](./apidoc/core.rst#Devices).

</div>

## The `Channels`

The third step to writing a Pulser program is [the selection of Channels among the Device](programming.md#pick-a-device).

As a reminder, the selection of a `Channel` defines the [interaction Hamiltonian](programming.md#interaction-hamiltonian) and [the driving Hamiltonian](programming.md#driving-hamiltonian) $H_D$.

$$
H^D(t) / \hbar = \frac{\Omega(t)}{2} e^{-j\phi} |a\rangle\langle b| + \frac{\Omega(t)}{2} e^{j\phi} |b\rangle\langle a| - \delta(t) |b\rangle\langle b|
$$

The `Channel`s available for selection are stored in the `channels` property of the `Device`, a dictionnary associating a name to a `Channel`. Let's have a look at the only channel available in `AnalogDevice`, the "rydberg_global" channel:

In [ ]:
import pulser

print(pulser.AnalogDevice.channels["rydberg_global"])

### Reading through the `Channel`'s specifications

The channel is characterized by:

- Its type, `Rydberg`, that defines the [states](conventions.md#bases) that can be addressed by the [driving Hamiltonian](programming.md#driving-hamiltonian) if this channel is picked. The `Rydberg` channel defines $\left|b\right>=\left|r\right>$ and $\left|a\right>=\left|g\right>$. All the type of channels can be found [here](conventions.md#bases).
- Its addressing, `Global`, which means that any Pulse added to this channel will implement the same driving Hamiltonian on all the atoms.
- Other parameters, that are going to set constraints on the quantities of the `Pulses` that can be added to the channel:
    - the **duration** of the pulse is constrained by the minimum and maximum pulse duration, as well as the clock period (it has to be a multiple of the clock period).
    - the **amplitude** is limited by the maximum amplitude.
    - the **detuning** is limited by the maximum absolute detuning.

<div class="alert alert-info">

**Note**:

The modulation bandwidth and the EOM support are associated with more advanced features explained [in this tutorial](./tutorials/output_mod_eom.nblink). For a full list of the parameters of a `Channel` and the constraints they set, please check the [API documentation](./apidoc/core.rst#channels).

</div>

## The `AnalogDevice`

In [ ]:
import pulser

print(pulser.AnalogDevice.specs)

The `pulser.AnalogDevice` only supports the $\left|r\right>$ and $\left|g\right>$ states, because it only supports one channel of type "Rydberg" (that can be declared using its name "rydberg_global"). It implements the [Ising Hamiltonian](programming.md#ising-hamiltonian): 

$$\frac{H}{\hbar}(t) = \sum_{i=1}^N \left (\frac{\Omega(t)}{2} e^{-j\phi(t)} |g\rangle\langle r|_i + \frac{\Omega(t)}{2} e^{j\phi(t)} |r\rangle\langle g|_i - \delta(t) |r\rangle\langle r|_i(t) + \sum_{j<i}\frac{C_6}{\hbar R_{ij}^6} \hat{n}_i \hat{n}_j \right)
$$

There:

- The maximum number of atoms $N$ is 25.
- The distance between the atoms $R_{ij}$ is at least 5µm and at most 70µm (twice the maximum distance). The distances are defined by placing the atoms in a 2D-plane, constrained by certain positions defined in a [layout](tutorials/reg_layouts.nblink).
- $C_6$ is the Ising interaction coefficient (865723.02).
- The "rydberg_global" being a "Global" channel, each pulse added to this channel is applied on all the atoms (the quantities $\Omega$, $\delta$, $\phi$ are the same for each atom).
- The Hamiltonian (and the pulses with their time-dependent quantities $\Omega$, $\delta$, $\phi$) can be defined between 0 and 4000 (the minimum of the maximum Sequence duration in the Device specifications and the `maximum_duration` of the channel).
- The value of $\Omega$ can go between 0 and `max_amp` and the value of $\delta$ can go between -`max_abs_detuning` and `max_abs_detuning`.